# ProRes Frames to Zarr Example
Here we write original ProRes compressed frames into a zarr array and show how to work with this database using lazy operations. Unfortunately this doesn't quite work at the moment. The zarr file containing encoded frames cannot be opened by Xarray.

#### Imports

In [ ]:
%matplotlib inline
import pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

#### Create a list of frames to analyze using the dbcamhd.json database

In [ ]:
dbcamhd = pd.read_json('dbcamhd.json', orient="records", lines=True)
dbcamhd.tail()

In [ ]:
fileindex = 2064
filename = dbcamhd.filename[fileindex]
frame_count = dbcamhd.frame_count[fileindex]
n_images = 10
frame_numbers = np.linspace(750,frame_count-6000, n_images, dtype=np.int64())
filename

In [ ]:
frame_numbers[0:10]

#### Create timestamps for frames

In [ ]:
from datetime import datetime

In [ ]:
timestamps = []
for i in range(len(frame_numbers)):
    timestamps.append(datetime.fromtimestamp(dbcamhd.timestamp[fileindex] + frame_numbers[i]/29.95))

In [ ]:
timestamps[0:5]

#### Create Xarray Dataset from Dask delayed functions
Here instead of calling get_frame, we call get_frame_data which returns the raw ProRes encoded frame. The encoded data has a compression ratio of about 10:1, so storage costs will be lower and transfer rates (in terms of frames/s) should be higher.

In [ ]:
from dask import delayed
import dask.array as da

In [ ]:
def get_frame_data_array(filename, frame_number, moov_atom=False):
    return np.array(camhd.get_frame_data(filename, frame_number, moov_atom), dtype='bytes')[None,]

In [ ]:
test_data = get_frame_data_array(filename, frame_numbers[4])

In [ ]:
plt.imshow(camhd.decode_frame_data(test_data, 'rgb24'));

In [ ]:
moov_atom = camhd.get_moov_atom(filename)
delayed_frame_data = []
for frame_number in frame_numbers:
    delayed_frame_data.append(da.from_delayed(
                              delayed(get_frame_data_array)(filename, frame_number, moov_atom),
                              shape=(1,), dtype='bytes'))

In [ ]:
delayed_frame_data[0]

In [ ]:
ds = xr.DataArray(da.concatenate(delayed_frame_data, axis=0),dims=['time'], coords={'time': timestamps}).to_dataset(name='video')
ds

In [ ]:
plt.imshow(camhd.decode_frame_data(ds.video[4].values, 'rgb24'));

#### Save to local zarr group

In [ ]:
ds.to_zarr('test.zarr', mode='w')

#### Open zarr group and plot image

In [ ]:
del ds
ds = xr.open_zarr('test.zarr')
ds

In [ ]:
plt.imshow(camhd.decode_frame_data(ds.video[4].values, 'rgb24'));